# Descripition

Simplified version with only one classifier and one regressor. Total ensemble models take too much time to train & tune.

 Idea credit to https://kaggle.com/competitions/equity-post-HCT-survival-predictions/writeups/agalar-2nd-place-solution

In [1]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=8e65520b59eb11280fa994cd4c211f391c483d6599b0ec3a79cfddd2f5b51b1d
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
import os
import pickle
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import optuna

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, CatBoostRegressor
from lifelines.utils import concordance_index
from numba import jit

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- Configuration ---
IS_LOCAL = False

if IS_LOCAL:
    BASE_DIR = './data/'
    WORK_DIR = './working/'
else:
    BASE_DIR = '/kaggle/input/equity-post-HCT-survival-predictions/'
    WORK_DIR = '/kaggle/working/'

DIRS = {
    'models': os.path.join(WORK_DIR, 'models'),
    'processed': os.path.join(WORK_DIR, 'processed_data'),
    'encoders': os.path.join(WORK_DIR, 'encoders'),
}

for d in DIRS.values():
    if not os.path.exists(d):
        os.makedirs(d)

print("Setup complete.")

Setup complete. Ready to roll!


# Metric

In [3]:
# C-index Computation
@jit(nopython=True)
def init_btree(vals):
    compare_times = np.empty_like(vals)
    last_row_idx = int(np.log2(len(vals) + 1) - 1)
    ragged_len = len(vals) - (2 ** (last_row_idx + 1) - 1)
    
    if ragged_len > 0:
        btm_idx = slice(None, 2 * ragged_len, 2)
        compare_times[-ragged_len:] = vals[btm_idx]
        vals = np.delete(vals, btm_idx)
    
    start = 0
    space = 2
    length = 2 ** last_row_idx
    
    while start < len(vals):
        compare_times[length - 1 : 2 * length - 1] = vals[start::space]
        start += int(space / 2)
        space *= 2
        length = int(length / 2)
    return compare_times

@jit(nopython=True)
def update_tree(counts, prediction, compare_times):
    i = 0
    n = len(compare_times)
    while i < n:
        current_val = compare_times[i]
        counts[i] += 1
        if prediction < current_val:
            i = 2 * i + 1
        elif prediction > current_val:
            i = 2 * i + 2
        else:
            return counts

@jit(nopython=True)
def get_rank(prediction, compare_times, counts):
    i = 0
    n = len(compare_times)
    rank = 0
    count = 0
    while i < n:
        current_val = compare_times[i]
        if prediction < current_val:
            i = 2 * i + 1
            continue
        elif prediction > current_val:
            rank += counts[i]
            next_i = 2 * i + 2
            if next_i < n:
                rank -= counts[next_i]
                i = next_i
                continue
            else:
                return rank, count
        else:
            count = counts[i]
            left_i = 2 * i + 1
            if left_i < n:
                left_count = counts[left_i]
                count -= left_count
                rank += left_count
                right_i = left_i + 1
                if right_i < n:
                    count -= counts[right_i]
            return rank, count
    return rank, count

@jit(nopython=True)
def process_tied_pairs(truth, preds, start_idx, compare_times, counts):
    next_idx = start_idx
    while next_idx < len(truth) and truth[next_idx] == truth[start_idx]:
        next_idx += 1
        
    pairs = counts[0] * (next_idx - start_idx)
    correct = np.int64(0)
    tied = np.int64(0)
    
    for i in range(start_idx, next_idx):
        r, c = get_rank(preds[i], compare_times, counts)
        correct += r
        tied += c
        
    return (pairs, correct, tied, next_idx)

@jit(nopython=True)
def fast_cindex_calc(times, pred_times, event_observed):
    died_mask = event_observed == 1
    
    died_truth = times[died_mask]
    sort_idx = np.argsort(died_truth)
    died_truth = died_truth[sort_idx]
    died_preds = pred_times[died_mask][sort_idx]
    
    censored_truth = times[~died_mask]
    sort_idx = np.argsort(censored_truth)
    censored_truth = censored_truth[sort_idx]
    censored_preds = pred_times[~died_mask][sort_idx]
    
    censored_ptr = 0
    died_ptr = 0
    
    # Initialize tree
    compare_times = init_btree(np.unique(died_preds))
    counts = np.full(len(compare_times), 0)
    
    n_pairs = np.int64(0)
    n_correct = np.int64(0)
    n_tied = np.int64(0)
    
    while True:
        has_censored = censored_ptr < len(censored_truth)
        has_died = died_ptr < len(died_truth)
        
        if has_censored and (not has_died or died_truth[died_ptr] > censored_truth[censored_ptr]):
            pairs, corr, tied, next_idx = process_tied_pairs(censored_truth, censored_preds, censored_ptr, compare_times, counts)
            censored_ptr = next_idx
            
        elif has_died and (not has_censored or died_truth[died_ptr] <= censored_truth[censored_ptr]):
            pairs, corr, tied, next_idx = process_tied_pairs(died_truth, died_preds, died_ptr, compare_times, counts)
            
            for p in died_preds[died_ptr:next_idx]:
                update_tree(counts, p, compare_times)
                
            died_ptr = next_idx
        else:
            break
            
        n_pairs += pairs
        n_correct += corr
        n_tied += tied
        
    return (n_correct + n_tied / 2) / n_pairs

def get_stratified_score(y, y_pred, events, race_groups):
    df = pd.DataFrame({'y': y, 'y_pred': y_pred, 'events': events, 'race': race_groups})
    scores = []
    
    for race_id, group_data in df.groupby('race'):
        c_idx = fast_cindex_calc(
            np.array(group_data['y']),
            np.array(group_data['y_pred']),
            np.array(group_data['events'])
        )
        scores.append(c_idx)
        
    mean_score = np.mean(scores)
    penalty = np.sqrt(np.var(scores))
    final_score = mean_score - penalty
    return float(final_score)

# Data Processing

In [4]:
def process_data(file_path, is_train=True):
    print(f"Processing {file_path}...")
    df = pd.read_csv(file_path)
    dict_df = pd.read_csv(os.path.join(BASE_DIR, 'data_dictionary.csv'))
    
    # Identify column types
    cat_cols = list(dict_df[dict_df.type == 'Categorical']['variable'])
    cat_cols = [c for c in cat_cols if c not in ['efs', 'efs_time']]
    
    num_cols = list(dict_df[dict_df.type == 'Numerical']['variable'])
    num_cols = [c for c in num_cols if c not in ['efs', 'efs_time']]
    
    # Create string versions of some numerical columns (treating them as categorical)
    for c in num_cols:
        if c not in ['donor_age', 'age_at_hct']:
            new_col = f"{c}_str"
            df[new_col] = df[c].astype(str)
            cat_cols.append(new_col)
            
    # Fill missing values
    features_cat = df[cat_cols].fillna('-1').astype(str)
    features_num = df[num_cols] # keep numerical as is
    
    # Combine them
    X = pd.concat([features_cat, features_num], axis=1)
    
    # Handle One-Hot Encoding
    if is_train:
        # For training, we fit the encoder
        oh_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        # We only OHE original categorical columns
        orig_cat_cols = [c for c in list(dict_df[dict_df.type == 'Categorical']['variable']) if c not in ('efs', 'efs_time')]
        
        oh_encoded = oh_encoder.fit_transform(df[orig_cat_cols])
        
        # Save the encoder for later
        with open(os.path.join(DIRS['encoders'], 'ohe.pkl'), 'wb') as f:
            pickle.dump({'features': orig_cat_cols, 'model': oh_encoder}, f)
    else:
        # For test, load the encoder
        with open(os.path.join(DIRS['encoders'], 'ohe.pkl'), 'rb') as f:
            saved = pickle.load(f)
        oh_encoder = saved['model']
        orig_cat_cols = saved['features']
        oh_encoded = oh_encoder.transform(df[orig_cat_cols])

    # Convert OHE result to dataframe
    oh_df = pd.DataFrame(oh_encoded, columns=[f'ohe_{i}' for i in range(oh_encoded.shape[1])])
    X = pd.concat([X, oh_df], axis=1)
    
    # Identify feature lists
    ohe_cols = list(oh_df.columns)
    
    result = {
        'X': X,
        'cat_cols': cat_cols,
        'num_cols': num_cols,
        'ohe_cols': ohe_cols,
        'ID': df['ID'],
        'race_group': df['race_group']
    }
    
    if is_train:
        result['efs'] = df['efs']
        result['efs_time'] = df['efs_time']
        
    return result

# --- Run the processing ---
train_data = process_data(os.path.join(BASE_DIR, 'train.csv'), is_train=True)
with open(os.path.join(DIRS['processed'], 'train_packed.pkl'), 'wb') as f:
    pickle.dump(train_data, f)

test_data = process_data(os.path.join(BASE_DIR, 'test.csv'), is_train=False)
with open(os.path.join(DIRS['processed'], 'test_packed.pkl'), 'wb') as f:
    pickle.dump(test_data, f)

Processing /kaggle/input/equity-post-HCT-survival-predictions/train.csv...
Processing /kaggle/input/equity-post-HCT-survival-predictions/test.csv...


# CatBoost Classifier

In [5]:
def prep_for_catboost(data_path):
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
    
    for c in data['cat_cols']:
        data['X'][c] = data['X'][c].astype('category')
        
    # Filter only columns we want
    use_cols = data['cat_cols'] + data['num_cols'] + data['ohe_cols']
    data['X'] = data['X'][use_cols]
    
    # Save category info to ensure consistency
    cats = {c: data['X'][c].cat.categories for c in data['X'].columns if data['X'][c].dtype == 'category'}
    return data, cats

def train_classifier(folds=5, seed=43):
    print("Training CatBoost Classifier...")
    data, cat_map = prep_for_catboost(os.path.join(DIRS['processed'], 'train_packed.pkl'))
    
    with open(os.path.join(DIRS['encoders'], 'cat_map_cls.pkl'), 'wb') as f:
        pickle.dump(cat_map, f)
        
    # Target: 1 if Event DID NOT happen (censored), 0 if it DID
    targets = (data['efs'] == 0).astype(int)
    
    params = {
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'cat_features': data['cat_cols'],
        'verbose': 500,
        'random_seed': seed,
        'depth': 6,
        'learning_rate': 0.02,
        'n_estimators': 2000,
        'l2_leaf_reg': 1,
        'subsample': 0.66,
        'colsample_bylevel': 0.8
    }
    
    model = CatBoostClassifier(**params)
    
    # Stratified K-Fold based on race and event status
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=888)
    split_target = data['efs'].astype(str) + '_' + data['race_group'].astype(str)
    
    oof_preds = np.zeros(len(data['X']))
    
    for fold_idx, (idx_tr, idx_val) in enumerate(skf.split(data['X'], split_target)):
        print(f"--- Fold {fold_idx} ---")
        X_tr, y_tr = data['X'].iloc[idx_tr], targets.iloc[idx_tr]
        X_val, y_val = data['X'].iloc[idx_val], targets.iloc[idx_val]
        
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            use_best_model=False
        )
        
        # Predict probability
        val_preds = model.predict_proba(X_val)[:, 1]
        oof_preds[idx_val] = val_preds
        
        # Save this fold's model
        payload = {'tr_idx': idx_tr, 'val_idx': idx_val, 'model': model}
        with open(os.path.join(DIRS['models'], f'catboost_cls_f{fold_idx}.pkl'), 'wb') as f:
            pickle.dump(payload, f)
            
    print(f"Overall OOF AUC: {roc_auc_score(targets, oof_preds)}")

# Run it
train_classifier()

Training CatBoost Classifier...
--- Fold 0 ---
0:	test: 0.6702781	best: 0.6702781 (0)	total: 212ms	remaining: 7m 2s
500:	test: 0.7565740	best: 0.7565740 (500)	total: 1m 2s	remaining: 3m 7s
1000:	test: 0.7610635	best: 0.7610635 (1000)	total: 2m 5s	remaining: 2m 4s
1500:	test: 0.7620212	best: 0.7621786 (1352)	total: 3m 7s	remaining: 1m 2s
1999:	test: 0.7621559	best: 0.7622603 (1666)	total: 4m 11s	remaining: 0us

bestTest = 0.7622603455
bestIteration = 1666

--- Fold 1 ---
0:	test: 0.6873330	best: 0.6873330 (0)	total: 159ms	remaining: 5m 18s
500:	test: 0.7539659	best: 0.7539659 (500)	total: 1m 2s	remaining: 3m 7s
1000:	test: 0.7580478	best: 0.7580677 (999)	total: 2m 6s	remaining: 2m 5s
1500:	test: 0.7591167	best: 0.7591718 (1489)	total: 3m 10s	remaining: 1m 3s
1999:	test: 0.7597379	best: 0.7598889 (1969)	total: 4m 13s	remaining: 0us

bestTest = 0.7598888507
bestIteration = 1969

--- Fold 2 ---
0:	test: 0.6697805	best: 0.6697805 (0)	total: 138ms	remaining: 4m 35s
500:	test: 0.7570486	best:

# LightGBM Regressor

In [6]:
def prep_for_lgb(data_path):
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
        
    use_cols = data['cat_cols'] + data['num_cols'] + data['ohe_cols']
    # Removing some specific features deemed noisy
    drops = ['year_hct_str', 'hla_match_a_high_str', 'hla_match_b_low_str', 'comorbidity_score_str']
    final_cols = [c for c in use_cols if f"{c}_str" not in drops and c not in drops] 
    
    X = data['X'].copy()
    drop_actual = []
    for d in drops:
        if d in X.columns: drop_actual.append(d)
    X = X.drop(drop_actual, axis=1)
    
    # Convert to category for LGBM
    for c in X.columns:
        if X[c].dtype == 'object':
            X[c] = X[c].astype('category')
            
    cats = {c: X[c].cat.categories for c in X.columns if X[c].dtype.name == 'category'}
    data['X'] = X
    return data, cats

def train_regressor(folds=5, seed=42):
    print("Training LightGBM Regressor...")
    data, cat_map = prep_for_lgb(os.path.join(DIRS['processed'], 'train_packed.pkl'))
    
    with open(os.path.join(DIRS['encoders'], 'cat_map_lgb.pkl'), 'wb') as f:
        pickle.dump(cat_map, f)
        
    # --- Custom Target Engineering ---
    # We rank the times separately for people who had the event vs those who didn't.
    norm_time = data['efs_time'].copy()
    mask_event = data['efs'] == 1
    mask_sensor = data['efs'] == 0
    
    norm_time[mask_event] = data['efs_time'][mask_event].rank() / mask_event.sum()
    norm_time[mask_sensor] = data['efs_time'][mask_sensor].rank() / mask_sensor.sum()
    
    # Weights: give more importance to actual events (0.6) vs censored (0.4)
    weights = np.where(mask_event, 0.6, 0.4)
    
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'learning_rate': 0.02,
        'num_iterations': 10000,
        'num_leaves': 128,
        'max_depth': 6,
        'min_child_samples': 20,
        'extra_trees': True,
        'max_bin': 128,
        'verbose': -1,
        'random_state': seed,
        'n_jobs': -1
    }
    
    model = lgb.LGBMRegressor(**params)
    
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=888)
    split_target = data['efs'].astype(str) + '_' + data['race_group'].astype(str)
    
    oof_preds = np.zeros(len(data['X']))
    
    for fold_idx, (idx_tr, idx_val) in enumerate(skf.split(data['X'], split_target)):
        print(f"--- Fold {fold_idx} ---")
        X_tr, y_tr = data['X'].iloc[idx_tr].copy(), norm_time.iloc[idx_tr]
        X_val, y_val = data['X'].iloc[idx_val].copy(), norm_time.iloc[idx_val]
        w_tr = weights[idx_tr]
        
        # Add 'efs' as a feature for training (cheat code for training, fixed for inference)
        X_tr['efs'] = data['efs'].iloc[idx_tr].astype('int').astype('category')
        X_val['efs'] = data['efs'].iloc[idx_val].astype('int').astype('category')
        
        # Focus evaluation only on events that actually happened
        mask_eval_event = data['efs'].iloc[idx_val] == 1
        
        model.fit(
            X_tr, y_tr,
            sample_weight=w_tr,
            eval_set=[(X_val[mask_eval_event], y_val[mask_eval_event])],
            callbacks=[]
        )
        
        oof_preds[idx_val] = model.predict(X_val)
        
        payload = {'tr_idx': idx_tr, 'val_idx': idx_val, 'model': model}
        with open(os.path.join(DIRS['models'], f'lgb_reg_f{fold_idx}.pkl'), 'wb') as f:
            pickle.dump(payload, f)
            
    c_idx = concordance_index(data['efs_time'][mask_event], oof_preds[mask_event], data['efs'][mask_event])
    print(f"OOF C-Index (Events only): {c_idx}")

train_regressor()

Training LightGBM Regressor...
--- Fold 0 ---
--- Fold 1 ---
--- Fold 2 ---
--- Fold 3 ---
--- Fold 4 ---
OOF C-Index (Events only): 0.7637697447664714


# Merge Two Models

In [7]:
def get_cls_preds(is_test=False, folds=5):
    # Load data
    fname = 'test_packed.pkl' if is_test else 'train_packed.pkl'
    data, _ = prep_for_catboost(os.path.join(DIRS['processed'], fname))
    
    # Restore categories
    with open(os.path.join(DIRS['encoders'], 'cat_map_cls.pkl'), 'rb') as f:
        cat_map = pickle.load(f)
    for c, cats in cat_map.items():
        if c in data['X'].columns:
            data['X'][c] = data['X'][c].astype('category').cat.set_categories(cats)
            
    preds = np.zeros(len(data['X']))
    
    for i in range(folds):
        with open(os.path.join(DIRS['models'], f'catboost_cls_f{i}.pkl'), 'rb') as f:
            pkg = pickle.load(f)
        
        model = pkg['model']
        if is_test:
            preds += model.predict_proba(data['X'])[:, 1] / folds
        else:
            # For OOF, only predict on validation set
            val_idx = pkg['val_idx']
            preds[val_idx] = model.predict_proba(data['X'].iloc[val_idx])[:, 1]
            
    return preds

def get_reg_preds(is_test=False, folds=5):
    fname = 'test_packed.pkl' if is_test else 'train_packed.pkl'
    data, _ = prep_for_lgb(os.path.join(DIRS['processed'], fname))
    
    with open(os.path.join(DIRS['encoders'], 'cat_map_lgb.pkl'), 'rb') as f:
        cat_map = pickle.load(f)
    for c, cats in cat_map.items():
        if c in data['X'].columns:
            data['X'][c] = data['X'][c].astype('category').cat.set_categories(cats)
            
    # CRITICAL: Set 'efs' to 1 for inference
    data['X']['efs'] = 1
    data['X']['efs'] = data['X']['efs'].astype('category')
    
    preds = np.zeros(len(data['X']))
    
    for i in range(folds):
        with open(os.path.join(DIRS['models'], f'lgb_reg_f{i}.pkl'), 'rb') as f:
            pkg = pickle.load(f)
        model = pkg['model']
        
        if is_test:
            preds += model.predict(data['X']) / folds
        else:
            val_idx = pkg['val_idx']
            preds[val_idx] = model.predict(data['X'].iloc[val_idx])
            
    return preds

def blend_preds(reg_p, cls_p, a, b, c):
    # reg_p = time prediction, cls_p = probability prediction
    term_y = (reg_p > 0) * c * np.abs(reg_p)**b
    term_x = (cls_p > 0) * np.abs(cls_p)**a
    
    # Merge Function
    combined = (1 - term_y) * term_x + term_y
    
    # Return ranks
    return pd.Series(combined).rank() / len(combined)

def optimize_blending():
    with open(os.path.join(DIRS['processed'], 'train_packed.pkl'), 'rb') as f:
        data = pickle.load(f)
    
    efs, times, race = data['efs'], data['efs_time'], data['race_group']
    
    # Get predictions
    p_cls = get_cls_preds(is_test=False)
    p_reg = get_reg_preds(is_test=False)
    
    # Optuna Objective
    def objective(trial, idx):
        a = trial.suggest_uniform('a', 2, 3.5)
        b = trial.suggest_uniform('b', 0.5, 1.5)
        c = trial.suggest_uniform('c', 0, 1)
        
        final_p = blend_preds(p_reg[idx], p_cls[idx], a, b, c)
        score = get_stratified_score(times[idx], final_p, efs[idx], race[idx])
        return score

    # Optimize per fold
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=888)
    split_target = efs.astype(str) + '_' + race.astype(str)
    
    best_params_list = []
    
    for i, (tr_idx, val_idx) in enumerate(skf.split(efs, split_target)):
        print(f"Optimizing blend for Fold {i}...")
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda t: objective(t, val_idx), n_trials=200)
        best_params_list.append(study.best_params)
        print(f"Best: {study.best_params}")
        
    return best_params_list

# Find best parameters
blend_params = optimize_blending()

Optimizing blend for Fold 0...
Best: {'a': 2.7922221319022764, 'b': 0.7698711256179719, 'c': 0.9127555081932986}
Optimizing blend for Fold 1...
Best: {'a': 2.737268527282331, 'b': 1.1942366971205687, 'c': 0.17762761044190378}
Optimizing blend for Fold 2...
Best: {'a': 3.374492437083786, 'b': 1.0165893153511152, 'c': 0.16606093106065467}
Optimizing blend for Fold 3...
Best: {'a': 2.327916207954, 'b': 1.1242979623333207, 'c': 0.43744448919508716}
Optimizing blend for Fold 4...
Best: {'a': 3.1334868282104167, 'b': 0.6146091190702917, 'c': 0.13841383528214324}


In [8]:
print("Generating Final Submission...")

test_cls = get_cls_preds(is_test=True)
test_reg = get_reg_preds(is_test=True)

final_blend = []
for params in blend_params:
    res = blend_preds(test_reg, test_cls, **params)
    final_blend.append(res)

avg_preds = np.mean(np.array(final_blend).T, axis=1)

submission = pd.DataFrame({
    'ID': test_data['ID'],
    'prediction': 1 - avg_preds
})

submission.to_csv('submission.csv', index=False)
print("Done! Submission saved.")
print(submission.head())

Generating Final Submission...
Done! Submission saved.
      ID  prediction
0  28800    0.333333
1  28801    0.666667
2  28802    0.000000
